## Step 1: Load Libraries

In [2]:
# Install all required packages
!pip install pandas krippendorff statsmodels scikit-learn seaborn matplotlib numpy


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [20]:
import pandas as pd
import krippendorff
from statsmodels.stats.inter_rater import fleiss_kappa
from sklearn.metrics import cohen_kappa_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats.inter_rater import aggregate_raters, fleiss_kappa
import itertools
import seaborn as sns
from scipy.stats import entropy
from sklearn.cluster import KMeans
from scipy.special import rel_entr
from scipy.stats import entropy
import itertools
from collections import Counter
import pandas as pd
import ast 
from statsmodels.stats.inter_rater import fleiss_kappa

## Step 2: Read and Prepare Data

In [12]:
df = pd.read_csv("./gold_dataset.csv")

In [13]:
df.head()

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.text-eng,Input.label,Input.parameter,Input.parameter_num_labels,Answer.manifesto_class.labels,Approve,Reject
0,39N6W9XWS05H65P9I0HAUP0E2RZGYY,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (152/152),100% (46/46),100% (46/46),"Madam President, Mr Posselt, I do not really t...",0,Par303_Governmental and Administrative Efficiency,2,[],NaN,NaN
1,39N6W9XWS05H65P9I0HAUP0E2RZGYY,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (238/239),100% (19/19),100% (19/19),"Madam President, Mr Posselt, I do not really t...",0,Par303_Governmental and Administrative Efficiency,2,[],NaN,NaN
2,39N6W9XWS05H65P9I0HAUP0E2RZGYY,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (287/287),100% (75/75),100% (19/19),"Madam President, Mr Posselt, I do not really t...",0,Par303_Governmental and Administrative Efficiency,2,[],NaN,NaN
3,3P7RGTLO71VLC9NPSLO9PVA06HDAKL,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (152/152),100% (46/46),100% (46/46),"Mr President, ladies and gentlemen, first of a...",1,Par405_Corporatism,2,"[""Keynesian Demand Management"",""Market regulat...",NaN,NaN
4,3P7RGTLO71VLC9NPSLO9PVA06HDAKL,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (287/287),100% (75/75),100% (19/19),"Mr President, ladies and gentlemen, first of a...",1,Par405_Corporatism,2,"[""Government and administrative efficiency"",""M...",NaN,NaN


#### **Data Cleaning and Pre-Proccessing**

Here, I want to keep HITId (represnting speeches), WorkerId, and Answer.manifesto_class.labels

In [14]:
# only keep needed columns and rename 
df_clean = df[['HITId', 'WorkerId', 'Answer.manifesto_class.labels']].copy() #remove unnecessary columns
df_clean.rename(columns={
    'HITId': 'speech_id',
    'WorkerId': 'worker_id',
    'Answer.manifesto_class.labels': 'answer'
}, inplace=True)

# extra cleaning, dropping nulls, making sure all labels are ints, 
df_clean = df_clean.sort_values(by=['speech_id', 'worker_id']).reset_index(drop=True)

# Final preview

df_clean

,speech_id,worker_id,answer
0,301KG0KXAZ355GRD9LSX6LBA2CHH2H,A1NBQ61Y6KO3O9,"[""Market regulation"",""Middle Class & Professio..."
1,301KG0KXAZ355GRD9LSX6LBA2CHH2H,A1X47COW2Y9SEL,"[""Market regulation""]"
2,301KG0KXAZ355GRD9LSX6LBA2CHH2H,A335R4YE2E34H6,"[""Market regulation""]"
3,302OLP89EMP3TBB1087B1CXS184CAA,A1NBQ61Y6KO3O9,"[""Government and administrative efficiency"",""E..."
4,302OLP89EMP3TBB1087B1CXS184CAA,A1X47COW2Y9SEL,"[""Economic Goals""]"
...,...,...,...
655,3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M,A1X47COW2Y9SEL,"[""Market regulation"",""Government and administr..."
656,3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M,A3GS2NTQ4XU059,"[""Free Market Economy""]"
657,3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN,A1BQ37ZGUM16XI,"[""Market regulation""]"
658,3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN,A1X47COW2Y9SEL,"[""Market regulation"",""Anti-Growth Economy""]"


define 'answer's

In [ ]:

# turn all the answers into an array 
all_answers = df_clean['answer'].dropna().apply(ast.literal_eval)
all_categories = [category for sublist in all_answers for category in sublist]
unique_categories = pd.Series(all_categories).unique()

print(unique_categories) #check 


['Market regulation' 'Middle Class & Professional Groups'
 'Government and administrative efficiency' 'Economic Planning'
 'Economic Goals' 'Anti-Growth Economy' 'Economic Growth'
 'Free Market Economy' 'Incentives' 'Keynesian Demand Management'
 'Corporatism / Mixed Economy' 'Technology and Infrastructure'
 'Labor Groups Positive' 'Labor Groups Negative']


Now, we compute a 'count' matrix in order to analyze similary with fleiss kappa

In [17]:
# Make sure 'answer' column is parsed as lists (from string format)
df_clean['parsed_answer'] = df_clean['answer'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) and x.strip() else [])

# Explode the answers so each category gets its own row
df_exploded = df_clean.explode('parsed_answer')

# Count how many times each category was selected per speech
df_counts = df_exploded.groupby(['speech_id', 'parsed_answer']).size().unstack(fill_value=0)

# Optional: sort columns alphabetically
df_counts = df_counts.sort_index(axis=1)

# Result: rows = speeches, columns = categories, values = counts
df_counts.head()

parsed_answer,Anti-Growth Economy,Corporatism / Mixed Economy,Economic Goals,Economic Growth,Economic Planning,Free Market Economy,Government and administrative efficiency,Incentives,Keynesian Demand Management,Labor Groups Negative,Labor Groups Positive,Market regulation,Middle Class & Professional Groups,Technology and Infrastructure
speech_id,,,,,,,,,,,,,,
301KG0KXAZ355GRD9LSX6LBA2CHH2H,0,0,0,0,0,0,0,0,0,0,0,3,1,0
302OLP89EMP3TBB1087B1CXS184CAA,0,0,2,0,1,0,1,0,0,0,0,0,0,0
306996CF7J2OPI6VQPINFRY7FTYB1E,1,0,0,1,1,0,1,0,0,0,0,1,0,0
3087LXLJ79YEVAXTZ80JKC6IPZKF0X,0,0,0,3,1,1,0,0,0,0,0,2,0,0
308KJXFUKEOKFSDCMU21QMXWPU4ATR,0,0,2,0,2,0,0,0,0,0,0,2,0,0
